# Template script for a strategy.
This includes
- Parsing the data from the Trader service into a pandas dataframe which is appropriate for creating technical indicators and producing trading signals
- Defining a standardized function to create all the technical indicators requried
- Defining a standardized function to create the signals across the time frame provided
- Defining a standardized function that returns the signals (if any) for the most recent time frame
- Defining a standardized function that returns the results of a backtest
- Defining a standardized function that returns the results of an optimization run

### 1. Parsing the data from the Trader service

In [1]:
import pandas as pd

def parse_data(json_data):
    df = pd.read_json(json_data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_values(by='date', inplace=True)
    df['Close'] = df['adj_close']
    df['Open'] = df['adj_open']
    df['High'] = df['adj_high']
    df['Low'] = df['adj_low']
    df['Volume'] = df['volume']
    return df


### 2. Create the technical indicators

In [9]:
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator, SMAIndicator


ema_window = 100
sma_window = 50
rsi_window = 14
bb_window = 14
bb_std = 1.5

def add_indicators(df, ema_window=ema_window, sma_window=sma_window, rsi_window=rsi_window, bb_window=bb_window, bb_std=bb_std):
    df['BB_high'] = BollingerBands(df['Close'], window=bb_window, window_dev=bb_std).bollinger_hband()
    df['BB_low'] = BollingerBands(df['Close'], window=bb_window, window_dev=bb_std).bollinger_lband()
    df['RSI'] = RSIIndicator(df['Close'], window=rsi_window).rsi()
    df['EMA'] = EMAIndicator(df['Close'], window=ema_window).ema_indicator()
    df['SMA'] = SMAIndicator(df['Close'], window=sma_window).sma_indicator()
    return df

# Test the function
# df = add_indicators(df)
# df.tail()

### 3. Create the signals
A signal needs to include all the information required to place an order:
- Type of trade: Buy or Sell
- Entry price limit
- Stop-loss
- Take-profit
- Expiry time of the order

In [3]:
def create_signals(df):
    df['Signal'] = ''
    df.loc[(df['Close'] > df['BB_high']) & (df['RSI'] > 50) & (df['SMA'] < df['EMA']), 'Signal'] = 'Sell'
    df.loc[(df['Close'] < df['BB_low']) & (df['RSI'] < 50) & (df['SMA'] > df['EMA']), 'Signal'] = 'Buy'
    return df

limit = 0.00
stop_loss = 0.2
take_profit_long = 2
take_profit_short = 0.5
quantity = 10

def process_buy(row):
    row['Limit']=row['Close'] * (1+limit)
    row['Stop_loss']=row['Close'] * (1-stop_loss)
    row['Quantity'] = quantity
    row['Take_profit'] = row['Close'] * (1 + take_profit_long)
    return row

def process_sell(row):
    row['Limit']=row['Close'] * (1-limit)
    row['Stop_loss']=row['Close'] * (1+stop_loss)
    row['Quantity'] = quantity
    row['Take_profit'] = row['Close'] * (1 - take_profit_short)
    return row

def add_trades(df):
    df['Limit'] = None
    df['Stop_loss'] = None
    df['Quantity'] = None
    df['Take_profit'] = None
    df['Expiry'] = None
    
    dfSignal = df[df['Signal'] != '']

    for index, row in dfSignal.iterrows():
        if row['Signal'] == 'Buy':
            df.loc[index] = process_buy(row)
        elif row['Signal'] == 'Sell':
            df.loc[index] = process_sell(row)

    return df


### Intermezzo - We need a helper plot function
This is for testing purposes only

In [4]:
import plotly.io as pio
pio.renderers.default = 'vscode'

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def add_buy_sell_points(df):
    df['sell_dp'] = None
    df['buy_dp'] = None
    df.loc[df['Signal'] == 'Sell', 'sell_dp'] = df['Limit']
    df.loc[df['Signal'] == 'Buy', 'buy_dp'] = df['Limit']
    return df

def create_plot(df):

    # dfpl = df[200:400].copy()
    dfpl = df.copy()
    
    #dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
    #dfpl.reset_index(inplace=True)
    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                    open=dfpl['Open'],
                    high=dfpl['High'],
                    low=dfpl['Low'],
                    close=dfpl['Close']),
                    go.Scatter(x=dfpl.index, y=dfpl['EMA'], line=dict(color='orange', width=2), name="EMA"),
                    go.Scatter(x=dfpl.index, y=dfpl['SMA'], line=dict(color='yellow', width=2), name="SMA"),
                    go.Scatter(x=dfpl.index, y=dfpl['BB_low'], line=dict(color='blue', width=1), name="BBL"),
                    go.Scatter(x=dfpl.index, y=dfpl['BB_high'], line=dict(color='blue', width=1), name="BBH")])
    
    dfpl = add_buy_sell_points(dfpl)

    # fig.add_scatter(x=dfpl.index, y=dfpl['Limit'], mode="markers",
    #                 marker=dict(size=10, color="darkorange"),
    #                 name="Signal")
    fig.add_scatter(x=dfpl.index, y=dfpl['sell_dp'], mode="markers",
                    marker=dict(size=10, color="darkorange"),
                    name="Sell")
    fig.add_scatter(x=dfpl.index, y=dfpl['buy_dp'], mode="markers",
                    marker=dict(size=10, color="darkgreen"),
                    name="Buy")
    return fig

### 4. Provide the signals for the most recent time frame

In [5]:
def get_current_signal_as_json(df):
    return df.last('1D').to_json(orient="records")

### 5. Wire up a backtesting function

In [6]:
from backtesting import Strategy, Backtest

# def bt_signal(values):
#     return values['Signal']

class Backtester(Strategy):

    def init(self):
        super().init()
        # self.bt_signal = self.I(bt_signal, self.data.df)

    def next(self):
        super().next()
        if self.data.Signal[-1] == 'Buy':
            # print('Buy!')
            if (self.position.is_short):
                # If we have a short position, close it
                self.position.close()

            if self.position.size == 0:
                self.buy(size=self.data.Quantity[-1], 
                        limit=self.data.Limit[-1], 
                        sl=self.data.Stop_loss[-1], 
                        tp=self.data.Take_profit[-1])
                
        elif self.data.Signal[-1] == 'Sell':
            # print('Sell!')

            test = self.data.Quantity[-1]
    
            if (self.position.is_long):
                # If we have a long position, close it
                self.position.close()
            
            if (self.position.size == 0):
                self.sell(size=self.data.Quantity[-1], 
                        limit=self.data.Limit[-1], 
                        sl=self.data.Stop_loss[-1], 
                        tp=self.data.Take_profit[-1])

cash = 10_000
commission = .00

def run_backtest(df):
    bt = Backtest(df, Backtester, cash=cash, commission=commission)
    return bt


### Testing it all so far

In [10]:
# Test the function
f = open('AAPL.json', 'r').read()
df = parse_data(f)
df = add_indicators(df)
df = create_signals(df)
df = add_trades(df)
# get_current_signal_as_json(df)
# test = run_backtest(df)
fig = create_plot(df)

In [11]:
fig.show()

In [9]:
test=run_backtest(df)

In [10]:
test.run()

Start                     2018-03-02 00:00...
End                       2023-03-01 00:00...
Duration                   1825 days 00:00:00
Exposure Time [%]                   51.589825
Equity Final [$]                  9839.941708
Equity Peak [$]                  10276.609621
Return [%]                          -1.600583
Buy & Hold Return [%]              246.476784
Return (Ann.) [%]                   -0.322697
Volatility (Ann.) [%]                2.490051
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -5.48529
Avg. Drawdown [%]                   -0.384577
Max. Drawdown Duration     1113 days 00:00:00
Avg. Drawdown Duration       75 days 00:00:00
# Trades                                   11
Win Rate [%]                        63.636364
Best Trade [%]                       53.46078
Worst Trade [%]                         -20.0
Avg. Trade [%]                    

In [11]:
test.plot()

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'



GridPlot(id='p1486', ...)